# Scaffold scoring refactor #132

## Setup

In [2]:
%load_ext autoreload
%autoreload 2
import hippo
from pathlib import Path
from mlog import setup_logger
import pandas as pd
import numpy as np
logger=setup_logger('notebook')

In [3]:
!cp -v CHIKV_prod3.sqlite scaffold_score.sqlite

'CHIKV_prod3.sqlite' -> 'scaffold_score.sqlite'


In [4]:
animal = hippo.HIPPO('scaffold_score', 'scaffold_score.sqlite')

Creating HIPPO animal
name = scaffold_score
db_path = scaffold_score.sqlite
DEBUG: hippo3.Database.__init__()
DEBUG: Database.path = scaffold_score.sqlite
DEBUG: hippo3.Database.connect()
DEBUG: sqlite3.version='2.6.0'
 SUCCESS  Database connected @ scaffold_score.sqlite!
 SUCCESS  Initialised animal HIPPO("scaffold_score")!


In [5]:
animal.db.delete_where(table='scaffold', key='scaffold_base = scaffold_superstructure')

In [41]:
# test compound set

bases = [93, 95, 100]

elabs = set()
for base in bases:
    base = animal.compounds[base]

    e = base.elabs

    if not e:
        continue

    print(base)
    
    ids = e[:30].ids
    for id in ids:
        elabs.add(id)

cset = animal.compounds[bases + list(elabs)]

C93
C95
C100


In [7]:
cset.risk_diversity

NotImplementedError: 

In [8]:
sql = f"""
WITH nums AS (
                    SELECT B.compound_id as base, A.compound_id AS elab, 
                    mol_num_hvyatms(A.compound_mol) - mol_num_hvyatms(B.compound_mol) AS diff 
                    FROM compound A, compound B
                    WHERE A.compound_base = B.compound_id
                    AND A.compound_id IN {cset.str_ids}
                )

SELECT * FROM nums
"""

c = animal.db.execute(sql)
print(c.fetchone())
print(c.fetchone())
print(c.fetchone())
print(c.fetchone())

None
None
None
None


In [42]:
sql = f"""
        SELECT COUNT(1) FROM scaffold
        WHERE scaffold_superstructure IN {cset.str_ids}
        GROUP BY scaffold_base
        """

c = animal.db.execute(sql)
print(c.fetchone())
print(c.fetchone())
print(c.fetchone())
print(c.fetchone())

(8,)
(30,)
(30,)
None


In [43]:
cset.elaboration_balance

-10.370899457402697

In [44]:
cset.num_bases_elaborated

3